In [70]:
from ipywidgets import interact
import pandas as pd
import sqlite3


conn = sqlite3.connect("IBL.db")

banks_id = [i[0] for i in conn.execute("SELECT id FROM banks;")]
years = [i[0] for i in conn.execute("SELECT DISTINCT(strftime('%Y', start_date)) FROM IBL ORDER BY start_date;")]

if (len(conn.execute("SELECT name FROM sqlite_master WHERE type='index' AND name LIKE 'idx%';").fetchall()) == 0):
    conn.execute("CREATE INDEX idx_IBL_bank_id ON IBL(bank_id)")
    conn.execute("CREATE INDEX idx_fact_IBL_id ON fact(IBL_id)")

@interact
def get_graph_IBL(bank_id = banks_id, year = years):
    acronym = conn.execute("SELECT acronym FROM banks WHERE id=?", (bank_id,)).fetchone()[0]

    plan = pd.DataFrame(
        conn.execute("""
                SELECT 
                    strftime('%m', plan_date) AS month, pl.amount AS plan, SUM(fc.amount) AS fact
                FROM  
                    plan_ as pl
                    LEFT JOIN fact as fc ON strftime('%m', fc.expences_date) = strftime('%m', pl.plan_date) 
                    LEFT JOIN IBL ON IBL.id = fc.IBL_id
                WHERE 
                    pl.bank_id = ? AND strftime('%Y', pl.plan_date) = ? AND IBL.bank_id = ?
                GROUP BY month;
                """, 
            (bank_id, year, bank_id)
            ), 
            columns=['month', 'plan', 'fact'],
    )
    if (len(plan) == 0):
        print("В этом году банк не пользовался кредитом")
    else:
        plan.plot(title=acronym, linewidth=3)


interactive(children=(Dropdown(description='bank_id', options=(1, 2), value=1), Dropdown(description='year', o…